# Imports

In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
from geopy.geocoders import Nominatim

In [4]:
import folium

# Cálculo da latitude e longitude

In [5]:
# Definição
cidades = ['Milão',
           'Bagnoregio',
           'Turim',
           'Verona',
           'Bergamo',
           'Lago di Como',
           'Bellagio',
           'Gênova',
           'Bolonha',
           'Brescia',
           'Veneza'
          ]
pais = 'Itália'

In [6]:
lista_cidades = []
lista_latitudes = []
lista_longitudes = []

# Encontrar lat-long de cada cidade
for cidade in cidades:
    
    geolocator = Nominatim(user_agent="carattaolivajulia@gmail.com")

    localizacao = geolocator.geocode(f"{cidade}, {pais}")

    # Verifica se a localização foi encontrada
    if localizacao:
        print(f"{cidade}: {(localizacao.latitude, localizacao.longitude)}")
        lista_cidades.append(cidade)
        lista_latitudes.append(localizacao.latitude)
        lista_longitudes.append(localizacao.longitude)
    else:
        print(f"Coordenadas de {cidade} não encontradas.")
        lista_cidades.append(cidade)
        lista_latitudes.append(None)
        lista_longitudes.append(None)

# DF de lat-long
df_lat_long = pd.DataFrame({'Cidade': lista_cidades, 'Latitude': lista_latitudes, 'Longitude': lista_longitudes})

Milão: (45.4641943, 9.1896346)
Bagnoregio: (42.62698, 12.0908718)
Turim: (45.0677551, 7.6824892)
Verona: (45.442497700000004, 10.985737689444763)
Bergamo: (45.756655699999996, 9.754219200862249)
Lago di Como: (45.9917589, 9.264881035005711)
Bellagio: (45.9872549, 9.2613001)
Gênova: (44.40726, 8.9338624)
Bolonha: (44.4938203, 11.3426327)
Brescia: (45.77958045, 10.4258729694612)
Veneza: (45.4371908, 12.3345898)


In [7]:
df_lat_long

,Cidade,Latitude,Longitude
0,Milão,45.464194,9.189635
1,Bagnoregio,42.626980,12.090872
2,Turim,45.067755,7.682489
3,Verona,45.442498,10.985738
4,Bergamo,45.756656,9.754219
5,Lago di Como,45.991759,9.264881
6,Bellagio,45.987255,9.261300
7,Gênova,44.407260,8.933862
8,Bolonha,44.493820,11.342633
9,Brescia,45.779580,10.425873


In [8]:
# Mapa centrado na média das coordenadas
mapa = folium.Map(location=[df_lat_long['Latitude'].mean(), 
                            df_lat_long['Longitude'].mean()], 
                            zoom_start=6, 
                            width='80%',  # Define a largura (pode ser percentual ou em pixels, ex: '500px')
                            height='80%'  # Define a altura (pode ser percentual ou em pixels, ex: '300px')
                 )

# Adiciona pontos de cada cidade
for _, row in df_lat_long.iterrows():
    folium.Marker(location=[row['Latitude'], row['Longitude']], popup=row['Cidade']).add_to(mapa)


mapa

# Cálculo da distância no globo

In [9]:
import math

def distancias_globo(lat1, lon1, lat2, lon2):
    
    # Converte latitude e longitude de graus para radianos
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    
    # Diferenças entre os pontos
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    
    # Fórmula de Haversine
    a = math.sin(dlat/2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    
    # Raio da Terra em km (pode mudar para milhas se necessário)
    R = 6371.0
    distance = R * c
    
    return distance

In [10]:
def calcular_distancias_em_df(df):
    
    distancias = []
    
    for i, origem in df.iterrows():
        
        for j, destino in df.iterrows():
            
            if i != j:
                distancia = distancias_globo(origem['Latitude'], origem['Longitude'], destino['Latitude'], destino['Longitude'])
                distancias.append({'Origem': origem['Cidade'], 'Destino': destino['Cidade'], 'Distancia_km': distancia})

    return pd.DataFrame(distancias)

In [11]:
df_distancias_globo = calcular_distancias_em_df(df_lat_long)

In [12]:
df_distancias_globo.sort_values(by='Distancia_km', ascending=True).head()

,Origem,Destino,Distancia_km
55,Lago di Como,Bellagio,0.572153
65,Bellagio,Lago di Como,0.572153
64,Bellagio,Bergamo,45.976449
45,Bergamo,Bellagio,45.976449
44,Bergamo,Lago di Como,46.027910


# Cálculo do tempo

In [13]:
df_distancias_globo['tempo_horas'] = round((df_distancias_globo['Distancia_km']/80),2)
df_distancias_globo['tempo_minutos'] = round((df_distancias_globo['Distancia_km']/80)*60,2)

df_distancias_globo.head()

,Origem,Destino,Distancia_km,tempo_horas,tempo_minutos
0,Milão,Bagnoregio,391.474460,4.89,293.61
1,Milão,Turim,125.916279,1.57,94.44
2,Milão,Verona,140.117681,1.75,105.09
3,Milão,Bergamo,54.645691,0.68,40.98
4,Milão,Lago di Como,58.952548,0.74,44.21


In [14]:
df_distancias_globo['menores_tempos'] = df_distancias_globo.sort_values(by=['Origem', 'tempo_minutos']).groupby('Origem').cumcount() + 1

In [15]:
df_distancias_globo[df_distancias_globo['Origem'] == 'Milão'].sort_values(by='tempo_minutos')

,Origem,Destino,Distancia_km,tempo_horas,tempo_minutos,menores_tempos
3,Milão,Bergamo,54.645691,0.68,40.98,1
5,Milão,Bellagio,58.427114,0.73,43.82,2
4,Milão,Lago di Como,58.952548,0.74,44.21,3
8,Milão,Brescia,102.335693,1.28,76.75,4
6,Milão,Gênova,119.237544,1.49,89.43,5
1,Milão,Turim,125.916279,1.57,94.44,6
2,Milão,Verona,140.117681,1.75,105.09,7
7,Milão,Bolonha,200.788192,2.51,150.59,8
9,Milão,Veneza,245.327400,3.07,184.00,9
0,Milão,Bagnoregio,391.474460,4.89,293.61,10


# Força bruta: cálculo de melhor rota
- Demora MUITO! Inviável

In [16]:
# import networkx as nx
# import itertools

# # Criar o grafo usando NetworkX
# G = nx.Graph()

# # Adicionar as cidades e as distâncias no grafo
# for _, row in df_distancias_globo.iterrows():
#     G.add_edge(row['Origem'], row['Destino'], weight=row['Distancia_km'])

# # Função para calcular o custo total de uma rota
# def calcular_custo_rota(G, rota):
#     custo_total = 0
#     for i in range(len(rota) - 1):
#         custo_total += nx.shortest_path_length(G, source=rota[i], target=rota[i+1], weight='weight')
#     # Retorna ao ponto inicial para fechar o ciclo
#     custo_total += nx.shortest_path_length(G, source=rota[-1], target=rota[0], weight='weight')
#     return custo_total

# # Função para encontrar a menor rota que passa por todas as cidades
# def menor_rota_tsp(G):
#     cidades = list(G.nodes)
#     menor_rota = None
#     menor_distancia = float('inf')

#     # Gerar todas as permutações possíveis das cidades
#     for rota in itertools.permutations(cidades):
#         distancia_atual = calcular_custo_rota(G, rota)
#         if distancia_atual < menor_distancia:
#             menor_distancia = distancia_atual
#             menor_rota = rota

#     return menor_rota, menor_distancia

In [17]:
# # Encontrar e imprimir a menor rota que passa por todas as cidades

# rota, distancia = menor_rota_tsp(G)
# print(f"A menor rota que passa por todas as cidades é: {rota}, com uma distância de {distancia}")

# Problema do Caixeiro Viajante (TSP): cálculo de melhor rota

## Cálculo da melhor rota

In [18]:
from pulp import *

In [19]:
df_lat_long

,Cidade,Latitude,Longitude
0,Milão,45.464194,9.189635
1,Bagnoregio,42.626980,12.090872
2,Turim,45.067755,7.682489
3,Verona,45.442498,10.985738
4,Bergamo,45.756656,9.754219
5,Lago di Como,45.991759,9.264881
6,Bellagio,45.987255,9.261300
7,Gênova,44.407260,8.933862
8,Bolonha,44.493820,11.342633
9,Brescia,45.779580,10.425873


In [20]:
# Criar uma matriz de distâncias
n = len(df_lat_long)
matriz_distancia_cidades = np.zeros((n, n))

# Preencher a matriz com as distâncias calculadas
for i in range(n):
    for j in range(n):
        if i != j:
            matriz_distancia_cidades[i][j] = distancias_globo(df_lat_long['Latitude'][i],
                                                              df_lat_long['Longitude'][i], 
                                                              df_lat_long['Latitude'][j], 
                                                              df_lat_long['Longitude'][j])

            cidade_i = df_lat_long['Cidade'][i]
            cidade_j = df_lat_long['Cidade'][j]

            # print(f'\n>> Iteração ({i}, {j}): ({cidade_i}, {cidade_j}). Distância: {matriz_distancia_cidades[i][j]}')

In [39]:
# Exibir a matriz de distâncias
# print(matriz_distancia_cidades)

In [22]:
matriz_distancia_cidades.shape

(11, 11)

In [23]:
n = len(matriz_distancia_cidades)

# Criar loop
rotas_individuais = [(i,j) for i in range(n) for j in range(n) if matriz_distancia_cidades[i,j] != 0.0]

print(rotas_individuais)

[(0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (0, 10), (1, 0), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (1, 10), (2, 0), (2, 1), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (2, 10), (3, 0), (3, 1), (3, 2), (3, 4), (3, 5), (3, 6), (3, 7), (3, 8), (3, 9), (3, 10), (4, 0), (4, 1), (4, 2), (4, 3), (4, 5), (4, 6), (4, 7), (4, 8), (4, 9), (4, 10), (5, 0), (5, 1), (5, 2), (5, 3), (5, 4), (5, 6), (5, 7), (5, 8), (5, 9), (5, 10), (6, 0), (6, 1), (6, 2), (6, 3), (6, 4), (6, 5), (6, 7), (6, 8), (6, 9), (6, 10), (7, 0), (7, 1), (7, 2), (7, 3), (7, 4), (7, 5), (7, 6), (7, 8), (7, 9), (7, 10), (8, 0), (8, 1), (8, 2), (8, 3), (8, 4), (8, 5), (8, 6), (8, 7), (8, 9), (8, 10), (9, 0), (9, 1), (9, 2), (9, 3), (9, 4), (9, 5), (9, 6), (9, 7), (9, 8), (9, 10), (10, 0), (10, 1), (10, 2), (10, 3), (10, 4), (10, 5), (10, 6), (10, 7), (10, 8), (10, 9)]


In [40]:
# Aplicar o caixeiro viajante para minimização da distância total da rota
tsp = LpProblem("RotaCidadesEspanha", LpMinimize)
tsp

RotaCidadesEspanha:
MINIMIZE
None
VARIABLES

In [26]:
# Variáveis de decisão

## Variáveis xij: Binárias e indicam se a rota entre as cidades i e j é incluída na solução.
## Variáveis ui: Contínuas e representam a ordem das cidades para ajudar a prevenir a formação de sub-ciclos.

x = LpVariable.dicts("x", rotas_individuais, cat='Binary')

u = LpVariable.dicts("u", [i for i in range(n)], lowBound=1, upBound=n, cat='Continuous') # Ordem que as rotas individuais são visitadas

In [28]:
# Função Objetivo: 
## Minimizar a soma total das distâncias entre as cidades, multiplicada pela variável binária que indica se a rota é parte da solução ou não
## Portanto, se uma rota é escolhida (x[i,j] = 1), a distância correspondente é adicionada à soma total

# Somar a função objetivo à função já existente no problema

tsp += lpSum([matriz_distancia_cidades[i,j] * x[i,j] for (i,j) in rotas_individuais])
# lpSum: Soma lista de expressões lineares

In [29]:
# Imprimir a função objetivo
print(">> Função Objetivo:\n")
print(tsp.objective)

>> Função Objetivo:

391.47445957004726*x_(0,_1) + 245.3273996262926*x_(0,_10) + 125.91627866066705*x_(0,_2) + 140.11768065976804*x_(0,_3) + 54.64569083356843*x_(0,_4) + 58.952547568885315*x_(0,_5) + 58.42711411959205*x_(0,_6) + 119.23754406512592*x_(0,_7) + 200.7881922946392*x_(0,_8) + 102.33569286761812*x_(0,_9) + 391.47445957004726*x_(1,_0) + 313.08755227373507*x_(1,_10) + 445.57123351231945*x_(1,_2) + 325.2880109754856*x_(1,_3) + 394.6868738576728*x_(1,_4) + 436.4477094159853*x_(1,_5) + 436.16990689872*x_(1,_6) + 322.4345066018239*x_(1,_7) + 216.15864882836948*x_(1,_8) + 374.81293832130365*x_(1,_9) + 245.3273996262926*x_(10,_0) + 313.08755227373507*x_(10,_1) + 366.4208632346686*x_(10,_2) + 105.23891412812857*x_(10,_3) + 203.87021439175385*x_(10,_4) + 246.1629686748689*x_(10,_5) + 246.31661526384718*x_(10,_6) + 291.1833696371473*x_(10,_7) + 130.74174281790116*x_(10,_8) + 153.27335252695278*x_(10,_9) + 125.91627866066705*x_(2,_0) + 445.57123351231945*x_(2,_1) + 366.4208632346686*x_(2

In [30]:
# Restrição 1: Garantir que cada cidade seja visitada apenas uma vez
for j in range(n):
    tsp += lpSum([x[i,j] for (i,m) in rotas_individuais if m==j]) == 1

# Restrição 2: Garantir que cada cidade seja partida exatamente uma vez
for i in range(n):
    tsp += lpSum([x[i,j] for (m,j) in rotas_individuais if m==i]) == 1

# Restrição 3: Eliminar subciclos
for (i,j) in rotas_individuais:
    if i > 0 and i != j:
        tsp += u[i] - u[j] + n*x[i,j] <= n-1

In [31]:
# Resolver o modelo
resolver_modelo = tsp.solve()
print(f'Status do problema: {LpStatus[resolver_modelo]}')

Status do problema: Optimal


In [32]:
# Inicializa uma lista para armazenar as variáveis
variaveis_selecionadas = []

for var in tsp.variables():
    if var.varValue > 0 and var.name.startswith('x'):
        variaveis_selecionadas.append([var.name])

# Cri DF de ordem
df_ordem_cidades = pd.DataFrame(variaveis_selecionadas, columns=['Ordem'])

In [33]:
# Extrair destino e origem
df_ordem_cidades[['ponto_origem', 'ponto_destino']] = df_ordem_cidades['Ordem'].str.extract(r'\((\d+),_(\d+)\)')
df_ordem_cidades['ponto_origem'] = df_ordem_cidades['ponto_origem'].astype(int)
df_ordem_cidades['ponto_destino'] = df_ordem_cidades['ponto_destino'].astype(int)

# Criar de-para de index
df_de_para = df_lat_long.reset_index().rename(columns={'index': 'index'})

# Usa a função map para substituir os números pelos nomes das cidades
df_ordem_cidades['Cidade_Origem'] = df_ordem_cidades['ponto_origem'].map(df_de_para.set_index('index')['Cidade'])
df_ordem_cidades['Cidade_Destino'] = df_ordem_cidades['ponto_destino'].map(df_de_para.set_index('index')['Cidade'])

In [34]:
df_ordem_cidades

,Ordem,ponto_origem,ponto_destino,Cidade_Origem,Cidade_Destino
0,"x_(0,_6)",0,6,Milão,Bellagio
1,"x_(1,_7)",1,7,Bagnoregio,Gênova
2,"x_(10,_8)",10,8,Veneza,Bolonha
3,"x_(2,_0)",2,0,Turim,Milão
4,"x_(3,_10)",3,10,Verona,Veneza
5,"x_(4,_9)",4,9,Bergamo,Brescia
6,"x_(5,_4)",5,4,Lago di Como,Bergamo
7,"x_(6,_5)",6,5,Bellagio,Lago di Como
8,"x_(7,_2)",7,2,Gênova,Turim
9,"x_(8,_1)",8,1,Bolonha,Bagnoregio


In [35]:
# Inicializar para armazenar
ordem_cidades = []
visitadas = set() 

# Ponto de partida
cidade_atual = df_ordem_cidades['Cidade_Origem'].iloc[0] 
cidade_destino = df_ordem_cidades['Cidade_Destino'].iloc[0]

# Loop
while cidade_atual and cidade_atual not in visitadas:

    # Registra ponto atual
    ordem_cidades.append(cidade_atual)
    visitadas.add(cidade_atual)

    # Atualiza ponto atual (origem)
    index_proxima_cidade_origem = df_ordem_cidades[df_ordem_cidades['Cidade_Origem'] == cidade_destino].index[0]
    cidade_atual = df_ordem_cidades['Cidade_Origem'].iloc[index_proxima_cidade_origem] 
    cidade_destino = df_ordem_cidades['Cidade_Destino'].iloc[index_proxima_cidade_origem]

# Cria uma lista de resultados 
resultado = [f'{i + 1}: {cidade}' for i, cidade in enumerate(ordem_cidades)]
print(f'Ordem de visita:\n')
for r in resultado:
    print(r)

# Cria DFs
df_ordem = pd.DataFrame([r.split(': ') for r in resultado], columns=['Ordem de visita', 'Cidade'])
df_aux_1 = pd.merge(df_ordem, df_lat_long, on='Cidade', how='left')

Ordem de visita:

1: Milão
2: Bellagio
3: Lago di Como
4: Bergamo
5: Brescia
6: Verona
7: Veneza
8: Bolonha
9: Bagnoregio
10: Gênova
11: Turim


In [36]:
def adicionar_distancias_proximo_ponto(df):

    distancias = []
    
    for i in range(len(df) - 1):
        origem = df.iloc[i]
        destino = df.iloc[i + 1]
        
        distancia = distancias_globo(origem['Latitude'], origem['Longitude'], destino['Latitude'], destino['Longitude'])
        distancias.append(distancia)
    
    # Para a última cidade, não há próximo ponto, então adicionamos NaN ou 0
    distancias.append(np.nan)  
    
    df['Distancia_ate_Proximo_Ponto_km'] = distancias
    
    return df

In [37]:
df_final = adicionar_distancias_proximo_ponto(df_aux_1)

df_final['Distancia_ate_Proximo_Ponto_km'] = df_final['Distancia_ate_Proximo_Ponto_km'].round(4)
df_final['tempo_horas'] = round((df_final['Distancia_ate_Proximo_Ponto_km']/80),2)
df_final['tempo_minutos'] = round((df_final['Distancia_ate_Proximo_Ponto_km']/80)*60,2)

df_final

,Ordem de visita,Cidade,Latitude,Longitude,Distancia_ate_Proximo_Ponto_km,tempo_horas,tempo_minutos
0,1,Milão,45.464194,9.189635,58.4271,0.73,43.82
1,2,Bellagio,45.987255,9.261300,0.5722,0.01,0.43
2,3,Lago di Como,45.991759,9.264881,46.0279,0.58,34.52
3,4,Bergamo,45.756656,9.754219,52.1594,0.65,39.12
4,5,Brescia,45.779580,10.425873,57.4571,0.72,43.09
5,6,Verona,45.442498,10.985738,105.2389,1.32,78.93
6,7,Veneza,45.437191,12.334590,130.7417,1.63,98.06
7,8,Bolonha,44.493820,11.342633,216.1586,2.70,162.12
8,9,Bagnoregio,42.626980,12.090872,322.4345,4.03,241.83
9,10,Gênova,44.407260,8.933862,123.1378,1.54,92.35


In [38]:
# Inicializar o mapa
mapa = folium.Map(location=[df_lat_long['Latitude'].mean(), df_lat_long['Longitude'].mean()], zoom_start=6, width='80%', height='80%') 

coordenadas = []

for i, cidade in enumerate(ordem_cidades):
    lat_long = df_lat_long[df_lat_long['Cidade'] == cidade]
    latitude = lat_long['Latitude'].values[0]
    longitude = lat_long['Longitude'].values[0]
    
    coordenadas.append([latitude, longitude])
    
    folium.Marker(
        location=[latitude, longitude], 
        popup=f"{i + 1}: {cidade}",
        icon=folium.DivIcon(html=f"""
            <div style="font-size: 8pt; color : black; 
                        background-color: white; border-radius: 50%; 
                        padding: 5px; text-align: center; width: 20px; height: 20px;">
                {i + 1}
            </div>"""),
        tooltip=f"{i + 1}: {cidade}" 
    ).add_to(mapa)

folium.PolyLine(coordenadas, color="blue",weight=5, opacity=0.7).add_to(mapa)

mapa